In [0]:
pip install azure.storage.blob
import requests
import pandas as pd
import json
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
from azure.core.pipeline.transport import HttpResponse
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, BooleanType, ArrayType
import gzip

In [0]:

def get_data(date, time):

    url = "https://data.gharchive.org/"+ str(date) +'-'+ str(time) +'.json.gz'
    print(url)

    response = requests.get(url)

    if response.status_code == 200:

       with open("cache.json.gz", "wb") as file:

            file.write(response.content)

       print("File downloaded successfully.")

    else:

       print("Error downloading the file.")

In [0]:
def unzip_jsongz():
    input_file = '/databricks/driver/cache.json.gz'
    output_file = '/databricks/driver/cache.json'
    with gzip.open(input_file, 'rb') as gz_file:
        with open(output_file, 'wb') as out_file:
            out_file.write(gz_file.read())

In [0]:
json_schema = StructType([
    StructField("id", StringType(), nullable=True),
    StructField("type", StringType(), nullable=True),
    StructField("actor", StructType([
        StructField("id", IntegerType(), nullable=True),
        StructField("login", StringType(), nullable=True),
        StructField("display_login", StringType(), nullable=True),
        StructField("gravatar_id", StringType(), nullable=True),
        StructField("url", StringType(), nullable=True),
        StructField("avatar_url", StringType(), nullable=True)
    ]), nullable=True),
    StructField("repo", StructType([
        StructField("id", IntegerType(), nullable=True),
        StructField("name", StringType(), nullable=True),
        StructField("url", StringType(), nullable=True)
    ]), nullable=True),
    StructField("payload", StringType(), nullable=True),
    StructField("public", BooleanType(), nullable=True),
    StructField("created_at", StringType(), nullable=True)
])

In [0]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [0]:
df = spark.read.json("file:///databricks/driver/cache.json")

In [0]:
df.show()

+--------------------+--------------------+-----------+--------------------+--------------------+------+--------------------+--------------------+
|               actor|          created_at|         id|                 org|             payload|public|                repo|                type|
+--------------------+--------------------+-----------+--------------------+--------------------+------+--------------------+--------------------+
|{https://avatars....|2023-04-04T15:00:04Z|28197453071|                null|{null, cc9abfdb6c...|  true|{623532634, appse...|           PushEvent|
|{https://avatars....|2023-04-04T15:00:04Z|28197453195|{https://avatars....|{started, null, n...|  true|{444995502, faceb...|          WatchEvent|
|{https://avatars....|2023-04-04T15:00:04Z|28197453255|                null|{null, 1098972a1d...|  true|{610768481, reyap...|           PushEvent|
|{https://avatars....|2023-04-04T15:00:04Z|28197453293|                null|{null, 51aa653149...|  true|{621998419, Sa

In [0]:
issues_df = df.filter(df.type == "IssuesEvent")

In [0]:
issues_df.show(5)

+--------------------+--------------------+-----------+--------------------+--------------------+------+--------------------+-----------+
|               actor|          created_at|         id|                 org|             payload|public|                repo|       type|
+--------------------+--------------------+-----------+--------------------+--------------------+------+--------------------+-----------+
|{https://avatars....|2023-04-04T15:00:05Z|28197454070|{https://avatars....|{opened, null, nu...|  true|{611687708, labtw...|IssuesEvent|
|{https://avatars....|2023-04-04T15:00:05Z|28197454170|{https://avatars....|{opened, null, nu...|  true|{440224087, Shopi...|IssuesEvent|
|{https://avatars....|2023-04-04T15:00:05Z|28197454447|{https://avatars....|{closed, null, nu...|  true|{279521289, WordP...|IssuesEvent|
|{https://avatars....|2023-04-04T15:00:06Z|28197455222|                null|{closed, null, nu...|  true|{619168929, dogwa...|IssuesEvent|
|{https://avatars....|2023-04-04T1

In [0]:
main_df = None
hours = [15,16]
for hour in hours:
    get_data("2023-04-04", hour)
    unzip_jsongz()
    df = spark.read.json("file:///databricks/driver/cache.json")
    issues_df = df.filter(df.type == "IssuesEvent")
    #df = lines_rdd.toDF().select(from_json("value", json_schema).alias("data")).select("data.*")
    if main_df is not None:
        main_df.unionAll(issues_df)
    else:
        main_df = issues_df

https://data.gharchive.org/2023-04-04-15.json.gz
File downloaded successfully.
https://data.gharchive.org/2023-04-04-16.json.gz
File downloaded successfully.


---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-3531875326276590>:9
      7 issues_df = df.filter(df.type == "IssuesEvent")
      8 if main_df is not None:
----> 9     main_df.unionAll(df)
     10 else:
     11     main_df = df

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/dataframe.py:3680, in DataFrame.unionAll(self, other)
   3648 def unionAll(self, other: "DataFrame") -> "DataFrame":
   3649     """Return a new :class:`DataFrame` containing union of rows in this and another
   3650     :class:`DataFrame

In [0]:
def upload_to_blob_storage( connection_string, container_name, file_path, blob_name):
    blob_service_client = BlobServiceClient.from_connection_string(connection_string)
    blob_client = blob_service_client.get_blob_client(container= container_name, blob= blob_name)

    with open(file_path, "rb") as data:
        blob_client.upload_blob(data)
     

In [0]:
from pathlib import Path

file_path = Path("/databricks/driver/cache.json")

if file_path.exists():
    print("File exists.")
else:
    print("File does not exist.")


File exists.
